# Sqlalchemy

In [1]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple/ SQLAlchemy==2.0.29

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/


In [2]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

In [3]:
# 创建数据库引擎
DATABASE_URL = "sqlite:///./test.db"
engine = create_engine(DATABASE_URL, echo=True)
Base = declarative_base()
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
conn = engine.connect()

In [4]:
from sqlalchemy import create_engine, Integer, String, Column
from sqlalchemy.orm import Mapped, mapped_column

class User(Base):
    __tablename__ = 'users'

    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    name: Mapped[str] = mapped_column(String, index=True)
    email: Mapped[str] = mapped_column(String, index=True)

Base.metadata.create_all(bind=engine)

2024-08-16 04:08:32,220 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 04:08:32,222 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2024-08-16 04:08:32,226 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-16 04:08:32,240 INFO sqlalchemy.engine.Engine COMMIT


## ORM

In [5]:
def create_user(name: str, email: str):
    db = SessionLocal()
    user = User(name=name, email=email)
    db.add(user)
    db.commit()
    db.refresh(user)
    db.close()
    return user

In [6]:
new_user = create_user(name="aaa", email="aa@test.com")
print(new_user.id)
print(new_user.name)
print(new_user.email)

2024-08-16 04:08:32,307 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 04:08:32,313 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES (?, ?)
2024-08-16 04:08:32,315 INFO sqlalchemy.engine.Engine [generated in 0.00208s] ('aaa', 'aa@test.com')
2024-08-16 04:08:32,323 INFO sqlalchemy.engine.Engine COMMIT
2024-08-16 04:08:32,332 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 04:08:32,391 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.email 
FROM users 
WHERE users.id = ?
2024-08-16 04:08:32,393 INFO sqlalchemy.engine.Engine [generated in 0.00206s] (1,)
2024-08-16 04:08:32,399 INFO sqlalchemy.engine.Engine ROLLBACK
1
aaa
aa@test.com


In [7]:
def get_user(user_id: int):
    db = SessionLocal()
    user = db.query(User).filter(User.id == user_id).first()
    db.close()
    return user

In [8]:
user = get_user(new_user.id)
print(user.id)
print(user.name)
print(user.email)

2024-08-16 04:08:32,424 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 04:08:32,428 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email 
FROM users 
WHERE users.id = ?
 LIMIT ? OFFSET ?
2024-08-16 04:08:32,430 INFO sqlalchemy.engine.Engine [generated in 0.00195s] (1, 1, 0)
2024-08-16 04:08:32,437 INFO sqlalchemy.engine.Engine ROLLBACK
1
aaa
aa@test.com


In [9]:
def update_user(user_id: int, name: str, email: str):
    db = SessionLocal()
    user = db.query(User).filter(User.id == user_id).first()
    if user:
        user.name = name
        user.email = email
        db.commit()
        db.refresh(user)
    db.close()
    return user

In [10]:
updated_user = update_user(user_id=new_user.id, name="bbb", email="bbb@test.com")
user = get_user(new_user.id)
print(user.id)
print(user.name)
print(user.email)

2024-08-16 04:08:32,517 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 04:08:32,520 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email 
FROM users 
WHERE users.id = ?
 LIMIT ? OFFSET ?
2024-08-16 04:08:32,521 INFO sqlalchemy.engine.Engine [cached since 0.0931s ago] (1, 1, 0)
2024-08-16 04:08:32,528 INFO sqlalchemy.engine.Engine UPDATE users SET name=?, email=? WHERE users.id = ?
2024-08-16 04:08:32,529 INFO sqlalchemy.engine.Engine [generated in 0.00138s] ('bbb', 'bbb@test.com', 1)
2024-08-16 04:08:32,535 INFO sqlalchemy.engine.Engine COMMIT
2024-08-16 04:08:32,593 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 04:08:32,595 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.email 
FROM users 
WHERE users.id = ?
2024-08-16 04:08:32,597 INFO sqlalchemy.engine.Engine [cached since 0.2061s ago] (1,)
2024-08-16 04:08:32,603 INFO sqlalchemy.engine.Engine ROLLBACK
2024-08-16 04:08:32,605 INFO sqla

In [11]:
def delete_user(user_id: int):
    db = SessionLocal()
    user = db.query(User).filter(User.id == user_id).first()
    if user:
        db.delete(user)
        db.commit()
    db.close()
    return user

In [12]:
delete_user(new_user.id)
user = get_user(new_user.id)
print(user)

2024-08-16 04:08:32,637 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 04:08:32,692 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email 
FROM users 
WHERE users.id = ?
 LIMIT ? OFFSET ?
2024-08-16 04:08:32,694 INFO sqlalchemy.engine.Engine [cached since 0.2655s ago] (1, 1, 0)
2024-08-16 04:08:32,702 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2024-08-16 04:08:32,704 INFO sqlalchemy.engine.Engine [generated in 0.00193s] (1,)
2024-08-16 04:08:32,710 INFO sqlalchemy.engine.Engine COMMIT
2024-08-16 04:08:32,720 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-16 04:08:32,721 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.email AS users_email 
FROM users 
WHERE users.id = ?
 LIMIT ? OFFSET ?
2024-08-16 04:08:32,725 INFO sqlalchemy.engine.Engine [cached since 0.2963s ago] (1, 1, 0)
2024-08-16 04:08:32,730 INFO sqlalchemy.engine.Engine ROLLBACK
None


## CORE

In [19]:
from sqlalchemy import insert

def create_user(name: str, email: str):
    data = {"name":name, "email":email}
    result = conn.execute(insert(User).values(data))
    print("sql_str----->", insert(User).values(data))
    return result
new_user_id = create_user(name="cc", email="cc@test.com")
print("id: {}".format(new_user_id.inserted_primary_key[0]))

2024-08-16 04:09:37,059 INFO sqlalchemy.engine.Engine INSERT INTO users (name, email) VALUES (?, ?)
2024-08-16 04:09:37,062 INFO sqlalchemy.engine.Engine [cached since 64.27s ago] ('cc', 'cc@test.com')
sql_str-----> INSERT INTO users (name, email) VALUES (:name, :email)
id: 1


In [20]:
from sqlalchemy import select

def get_user(user_id: int):
    filters = {"id": user_id}
    result = conn.execute(select(User).filter_by(**filters)).first()
    # result = result.scalars().first()
    print("sql_str----->", select(User).filter_by(**filters))
    return result

user = get_user(user_id=new_user_id.inserted_primary_key[0])
print("获取用户信息: {}".format(user))

2024-08-16 04:09:38,493 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.email 
FROM users 
WHERE users.id = ?
2024-08-16 04:09:38,497 INFO sqlalchemy.engine.Engine [cached since 65.67s ago] (1,)
sql_str-----> SELECT users.id, users.name, users.email 
FROM users 
WHERE users.id = :id_1
获取用户信息: (1, 'cc', 'cc@test.com')


In [21]:
from sqlalchemy import update

def update_user(user_id: int, name: str, email: str):
    filters = {"id":user_id}
    data = {"name":name, "email":email}
    result = conn.execute(update(User).filter_by(**filters).values(**data))
    print("sql_str----->", update(User).filter_by(**filters).values(**data))
    return result

update_user = update_user(user_id=new_user_id.inserted_primary_key[0], name="dd", email="dd@test.com")
user = get_user(user_id=new_user_id.inserted_primary_key[0])
print("更新后的用户信息: {}".format(user))

2024-08-16 04:09:42,375 INFO sqlalchemy.engine.Engine UPDATE users SET name=?, email=? WHERE users.id = ?
2024-08-16 04:09:42,376 INFO sqlalchemy.engine.Engine [cached since 69.47s ago] ('dd', 'dd@test.com', 1)
sql_str-----> UPDATE users SET name=:name, email=:email WHERE users.id = :id_1
2024-08-16 04:09:42,381 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.email 
FROM users 
WHERE users.id = ?
2024-08-16 04:09:42,382 INFO sqlalchemy.engine.Engine [cached since 69.56s ago] (1,)
sql_str-----> SELECT users.id, users.name, users.email 
FROM users 
WHERE users.id = :id_1
更新后的用户信息: (1, 'dd', 'dd@test.com')


In [22]:
from sqlalchemy import delete

def delete_user(user_id: int):
    filters = {"id":user_id}
    result = conn.execute(delete(User).filter_by(**filters))
    print("sql_str----->", delete(User).filter_by(**filters))
    return result

delete_user = delete_user(user_id=new_user_id.inserted_primary_key[0])
user = get_user(user_id=new_user_id.inserted_primary_key[0])
print("更新后的用户信息: {}".format(user))

2024-08-16 04:09:45,311 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2024-08-16 04:09:45,313 INFO sqlalchemy.engine.Engine [cached since 72.38s ago] (1,)
sql_str-----> DELETE FROM users WHERE users.id = :id_1
2024-08-16 04:09:45,318 INFO sqlalchemy.engine.Engine SELECT users.id, users.name, users.email 
FROM users 
WHERE users.id = ?
2024-08-16 04:09:45,319 INFO sqlalchemy.engine.Engine [cached since 72.5s ago] (1,)
sql_str-----> SELECT users.id, users.name, users.email 
FROM users 
WHERE users.id = :id_1
更新后的用户信息: None
